In [1]:
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
 
register_matplotlib_converters()
%matplotlib inline
sns.set_style('whitegrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def add_hours_dam(df, start_hour, interval=48):
    for i in range(start_hour, start_hour+interval):
        start = str(i)
        df["DAM LMP_PRC_"+start+"_hours_ago"] = 0.0
    for i in range(len(df)-1, interval-1, -1):
        for j in range(start_hour, start_hour+interval):
            start = str(j)
            v = df['DAM LMP_PRC'].iloc[i-j,] 
            df.at[i, "DAM LMP_PRC_"+start+"_hours_ago"] =  v
    return df


def add_hours_rtm(df, start_hour, interval=48):
    for rtm in range(1, 13):
        for i in range(start_hour, start_hour+interval):
            start = str(i)
            df["RTM_"+str(rtm) +"_LMP_PRC_"+start+"_hours_ago"] = 0.0
        for i in range(len(df)-1, interval-1, -1):
            for j in range(start_hour, start_hour+interval):
                start = str(j)
                v = df[('RTM LMP_PRC', rtm)].iloc[i-j,] 
                df.at[i, "RTM_"+str(rtm) +"_LMP_PRC_"+start+"_hours_ago"] =  v
    return df

def add_hours_gap(df, start_hour, interval=48):
    for i in range(start_hour, start_hour+interval):
        start = str(i)
        df["GAP LMP_PRC_"+start+"_hours_ago"] = 0.0
    for i in range(len(df)-1, interval-1, -1):
        for j in range(start_hour, start_hour+interval):
            start = str(j)
            v = df['Gap'].iloc[i-j,] 
            df.at[i, "GAP LMP_PRC_"+start+"_hours_ago"] =  v
    return df

In [2]:
def Evaluate_Scaled(Y_true, predictions):
    print('rMAE '+str(r_MAE(Y_true, predictions)*100) +'%')
    print('rRMSE '+str(r_RMSE(Y_true, predictions)*100)+'%')
    print('MAPE '+str(MAPE(Y_true, predictions))+'%')

def Evaluate(Y_true, predictions):
    print('MAE '+str(MAE(Y_true, predictions)))
    print('RMSE '+str(RMSE(Y_true, predictions)))

In [3]:
from sklearn.metrics import mean_absolute_error

def MAE(true, predicted):
    return mean_absolute_error(true, predicted)

def r_MAE(true, predicted):
    return mean_absolute_error(true, predicted)/np.mean(true)

def MSE(true, predicted):
    squared_diff = np.square(true - predicted)
    return np.mean(squared_diff)

# Root Mean Square Error
def RMSE(true, predicted):
    squared_diff = np.square(true - predicted)    
    return np.sqrt(np.mean(squared_diff))

# R-squared, coefficient of determination
def R_squared(true, predicted):
    true      = np.array(true)
    predicted = np.array(predicted)
    sum_squared_diff = sum(np.square(true - predicted))
    variance  = sum(np.square(true - np.mean(true)))
    calc_r2   = 1 - (sum_squared_diff / variance)
    return calc_r2

def r_MSE(true, predicted):
    mse = MSE(true, predicted)
    return mse/np.mean(true)

def r_RMSE(true, predicted):
    rmse = RMSE(true, predicted)
    return rmse/np.mean(true)


def MAPE(true, predicted): 
    return np.mean(np.abs((true - predicted) / true)) * 100

In [4]:
feature_names = ['DAM LMP_PRC_1_hours_ago',
 'GAP LMP_PRC_24_hours_ago',
 'RTM_9_LMP_PRC_25_hours_ago',
 'DAM LMP_PRC_24_hours_ago',
 'RTM_9_LMP_PRC_23_hours_ago',
 'RTM_8_LMP_PRC_54_hours_ago',
 'RTM_2_LMP_PRC_33_hours_ago',
 'RTM_2_LMP_PRC_24_hours_ago',
 'RTM_7_LMP_PRC_54_hours_ago',
 'RTM_10_LMP_PRC_59_hours_ago',
 'RTM_11_LMP_PRC_38_hours_ago',
 'GAP LMP_PRC_58_hours_ago',
 'RTM_12_LMP_PRC_49_hours_ago',
 'DAM LMP_PRC_23_hours_ago',
 'DAM LMP_PRC_20_hours_ago',
 'RTM_12_LMP_PRC_38_hours_ago',
 'RTM_9_LMP_PRC_57_hours_ago',
 'Demand_Forecast_Day_Ahead',
 'RTM_11_LMP_PRC_21_hours_ago',
 'RTM_2_LMP_PRC_23_hours_ago',
 'RTM_4_LMP_PRC_25_hours_ago',
 'GAP LMP_PRC_50_hours_ago',
 'DAM LMP_PRC_44_hours_ago',
 'GAP LMP_PRC_47_hours_ago',
 'Solar Zenith Angle',
 'RTM_1_LMP_PRC_24_hours_ago',
 'GAP LMP_PRC_23_hours_ago',
 'RTM_3_LMP_PRC_59_hours_ago',
 'RTM_10_LMP_PRC_27_hours_ago',
 'GAP LMP_PRC_33_hours_ago',
 'DAM LMP_PRC_10_hours_ago',
 'RTM_2_LMP_PRC_30_hours_ago',
 'DAM LMP_PRC_47_hours_ago',
 'RTM_2_LMP_PRC_54_hours_ago',
 'GAP LMP_PRC_46_hours_ago',
 'RTM_11_LMP_PRC_14_hours_ago',
 'DAM LMP_PRC_9_hours_ago',
 'RTM_12_LMP_PRC_22_hours_ago',
 'RTM_2_LMP_PRC_43_hours_ago',
 'Relative Humidity',
 'RTM_9_LMP_PRC_54_hours_ago',
 'RTM_10_LMP_PRC_50_hours_ago',
 'RTM_12_LMP_PRC_25_hours_ago',
 'sea_level_pressure_set_1d',
 'GAP LMP_PRC_55_hours_ago',
 'GAP LMP_PRC_43_hours_ago',
 'RTM_1_LMP_PRC_46_hours_ago',
 'GAP LMP_PRC_42_hours_ago',
 'GAP LMP_PRC_26_hours_ago',
 'Wind Direction',
 'cloud_layer_3_code_set_1',
 ('RT_Forecast_RTD', 2),
 'Precipitable Water',
 'GAP LMP_PRC_49_hours_ago',
 'RTM_11_LMP_PRC_55_hours_ago',
 ('RT_Forecast_RTD', 1),
 'GAP LMP_PRC_45_hours_ago',
 'RTM_3_LMP_PRC_21_hours_ago',
 'RTM_7_LMP_PRC_33_hours_ago',
 'GAP LMP_PRC_56_hours_ago',
 'Fill Flag',
 'humidity',
 'RTM_12_LMP_PRC_54_hours_ago',
 'RTM_7_LMP_PRC_57_hours_ago',
 'RTM_4_LMP_PRC_24_hours_ago',
 'RTM_12_LMP_PRC_21_hours_ago',
 'RTM_5_LMP_PRC_45_hours_ago',
 'RTM_9_LMP_PRC_17_hours_ago',
 'GAP LMP_PRC_17_hours_ago',
 'RTM_9_LMP_PRC_47_hours_ago',
 'RTM_5_LMP_PRC_14_hours_ago',
 'RTM_1_LMP_PRC_58_hours_ago',
 'GAP LMP_PRC_12_hours_ago',
 'RTM_12_LMP_PRC_48_hours_ago',
 'GAP LMP_PRC_37_hours_ago',
 'RTM_6_LMP_PRC_17_hours_ago',
 'RTM_2_LMP_PRC_46_hours_ago',
 'windspeed',
 'RTM_9_LMP_PRC_30_hours_ago',
 'Demand_Forecast_2Day_Ahead',
 'RTM_1_LMP_PRC_25_hours_ago',
 'DAM LMP_PRC_4_hours_ago',
 'RTM_3_LMP_PRC_25_hours_ago',
 'RTM_5_LMP_PRC_49_hours_ago',
 'RTM_4_LMP_PRC_20_hours_ago',
 'GAP LMP_PRC_44_hours_ago',
 'RTM_6_LMP_PRC_14_hours_ago',
 'GAP LMP_PRC_51_hours_ago',
 'GAP LMP_PRC_18_hours_ago',
 'RTM_8_LMP_PRC_27_hours_ago',
 'RTM_7_LMP_PRC_21_hours_ago',
 'DAM LMP_PRC_48_hours_ago',
 'dewpoint',
 'DAM LMP_PRC_42_hours_ago',
 'RTM_12_LMP_PRC_26_hours_ago',
 'RTM_7_LMP_PRC_52_hours_ago',
 'RTM_8_LMP_PRC_14_hours_ago',
 'Temperature',
 'RTM_11_LMP_PRC_48_hours_ago',
 'RTM_1_LMP_PRC_26_hours_ago',
 'RTM_12_LMP_PRC_37_hours_ago',
 ('RT_Forecast_RTD', 3),
 'RTM_1_LMP_PRC_33_hours_ago',
 'GAP LMP_PRC_25_hours_ago',
 'DAM LMP_PRC_45_hours_ago',
 'DAM LMP_PRC_22_hours_ago',
 'RTM_12_LMP_PRC_47_hours_ago',
 'RTM_9_LMP_PRC_39_hours_ago',
 'DAM LMP_PRC_21_hours_ago',
 'RTM_7_LMP_PRC_27_hours_ago',
 'RTM_11_LMP_PRC_23_hours_ago',
 'RTM_5_LMP_PRC_17_hours_ago',
 'DAM LMP_PRC_39_hours_ago',
 'RTM_11_LMP_PRC_31_hours_ago',
 ('DAM RT_Forecast_RTPD', 2),
 'RTM_10_LMP_PRC_38_hours_ago',
 'GAP LMP_PRC_59_hours_ago',
 'RTM_6_LMP_PRC_13_hours_ago',
 'RTM_7_LMP_PRC_26_hours_ago',
 'Cloud Type',
 'DAM LMP_PRC_46_hours_ago',
 'RTM_9_LMP_PRC_33_hours_ago',
 'RTM_7_LMP_PRC_45_hours_ago',
 'RTM_11_LMP_PRC_22_hours_ago',
 'RTM_8_LMP_PRC_24_hours_ago',
 'RTM_11_LMP_PRC_16_hours_ago',
 ('RT_Forecast_RTD', 4),
 'RTM_3_LMP_PRC_51_hours_ago',
 'RTM_7_LMP_PRC_24_hours_ago',
 'RTM_1_LMP_PRC_17_hours_ago',
 'ceiling_set_1',
 'RTM_9_LMP_PRC_53_hours_ago',
 'RTM_1_LMP_PRC_32_hours_ago',
 'RTM_10_LMP_PRC_53_hours_ago',
 'RTM_3_LMP_PRC_23_hours_ago',
 'RTM_11_LMP_PRC_18_hours_ago',
 'RTM_8_LMP_PRC_29_hours_ago',
 'RTM_12_LMP_PRC_41_hours_ago',
 'RTM_5_LMP_PRC_21_hours_ago',
 'GAP LMP_PRC_57_hours_ago',
 'RTM_2_LMP_PRC_59_hours_ago',
 'GAP LMP_PRC_34_hours_ago',
 'RTM_5_LMP_PRC_32_hours_ago',
 'RTM_7_LMP_PRC_53_hours_ago',
 'RTM_9_LMP_PRC_35_hours_ago',
 'GHI',
 'RTM_10_LMP_PRC_55_hours_ago',
 'RTM_6_LMP_PRC_24_hours_ago',
 'RTM_1_LMP_PRC_15_hours_ago',
 'RTM_6_LMP_PRC_57_hours_ago',
 'GAP LMP_PRC_48_hours_ago',
 'RTM_4_LMP_PRC_16_hours_ago',
 'RTM_2_LMP_PRC_25_hours_ago',
 'GAP LMP_PRC_28_hours_ago',
 'GAP LMP_PRC_15_hours_ago',
 'DAM LMP_PRC_6_hours_ago',
 'RTM_6_LMP_PRC_21_hours_ago',
 'RTM_1_LMP_PRC_43_hours_ago',
 'RTM_4_LMP_PRC_32_hours_ago',
 'RTM_10_LMP_PRC_47_hours_ago',
 'RTM_9_LMP_PRC_44_hours_ago',
 'DAM LMP_PRC_5_hours_ago',
 'RTM_4_LMP_PRC_21_hours_ago',
 'RTM_11_LMP_PRC_29_hours_ago',
 'RTM_12_LMP_PRC_44_hours_ago',
 'DAM LMP_PRC_15_hours_ago',
 'DAM LMP_PRC_2_hours_ago',
 'RTM_8_LMP_PRC_17_hours_ago',
 'RTM_10_LMP_PRC_24_hours_ago',
 'RTM_1_LMP_PRC_34_hours_ago',
 'RTM_4_LMP_PRC_54_hours_ago',
 'RTM_10_LMP_PRC_35_hours_ago',
 'RTM_8_LMP_PRC_57_hours_ago',
 'GAP LMP_PRC_38_hours_ago',
 'RTM_3_LMP_PRC_40_hours_ago',
 'RTM_8_LMP_PRC_39_hours_ago',
 'RTM_12_LMP_PRC_45_hours_ago',
 'RTM_8_LMP_PRC_41_hours_ago',
 'RTM_9_LMP_PRC_55_hours_ago',
 'DAM LMP_PRC_29_hours_ago',
 'RTM_3_LMP_PRC_56_hours_ago',
 'RTM_5_LMP_PRC_33_hours_ago',
 'DAM LMP_PRC_25_hours_ago',
 'RTM_3_LMP_PRC_19_hours_ago',
 'Hour',
 'DAM LMP_PRC_12_hours_ago',
 'RTM_9_LMP_PRC_50_hours_ago',
 'RTM_11_LMP_PRC_15_hours_ago',
 'RTM_10_LMP_PRC_29_hours_ago',
 'RTM_1_LMP_PRC_13_hours_ago',
 'relative_humidity_set_1',
 'RTM_4_LMP_PRC_23_hours_ago',
 'RTM_7_LMP_PRC_44_hours_ago',
 'RTM_4_LMP_PRC_42_hours_ago',
 'RTM_12_LMP_PRC_39_hours_ago',
 'RTM_5_LMP_PRC_20_hours_ago',
 'RTM_7_LMP_PRC_25_hours_ago',
 'RTM_12_LMP_PRC_40_hours_ago',
 'RTM_6_LMP_PRC_18_hours_ago',
 'RTM_9_LMP_PRC_19_hours_ago',
 'DHI',
 'RTM_12_LMP_PRC_57_hours_ago',
 'RTM_11_LMP_PRC_27_hours_ago',
 'RTM_7_LMP_PRC_23_hours_ago',
 'RTM_8_LMP_PRC_49_hours_ago',
 'GAP LMP_PRC_21_hours_ago',
 ('RT_Forecast_RTD', 6),
 'RTM_8_LMP_PRC_55_hours_ago',
 'RTM_10_LMP_PRC_20_hours_ago',
 'RTM_3_LMP_PRC_42_hours_ago',
 'RTM_10_LMP_PRC_34_hours_ago',
 'RTM_10_LMP_PRC_14_hours_ago',
 'RTM_6_LMP_PRC_26_hours_ago',
 'RTM_1_LMP_PRC_21_hours_ago',
 'Day',
 'RTM_2_LMP_PRC_44_hours_ago',
 'GAP LMP_PRC_20_hours_ago',
 'RTM_4_LMP_PRC_55_hours_ago',
 'pressure',
 'RTM_3_LMP_PRC_26_hours_ago',
 'RTM_8_LMP_PRC_13_hours_ago',
 'GAP LMP_PRC_54_hours_ago',
 'RTM_10_LMP_PRC_18_hours_ago',
 'RTM_3_LMP_PRC_33_hours_ago',
 'RTM_11_LMP_PRC_46_hours_ago',
 'RTM_3_LMP_PRC_35_hours_ago',
 'RTM_7_LMP_PRC_46_hours_ago',
 'GAP LMP_PRC_29_hours_ago',
 'RTM_9_LMP_PRC_22_hours_ago',
 'RTM_5_LMP_PRC_36_hours_ago',
 'RTM_4_LMP_PRC_46_hours_ago',
 'RTM_10_LMP_PRC_22_hours_ago',
 'RTM_6_LMP_PRC_56_hours_ago',
 'RTM_10_LMP_PRC_30_hours_ago',
 'RTM_10_LMP_PRC_39_hours_ago',
 ('RT_Forecast_RTD', 10),
 'GAP LMP_PRC_30_hours_ago',
 'GAP LMP_PRC_16_hours_ago',
 'RTM_4_LMP_PRC_17_hours_ago',
 'RTM_7_LMP_PRC_34_hours_ago',
 ('RT_Forecast_RTD', 8),
 'RTM_4_LMP_PRC_47_hours_ago',
 'weather_condition_set_1d',
 'RTM_7_LMP_PRC_14_hours_ago',
 'RTM_4_LMP_PRC_53_hours_ago',
 'RTM_11_LMP_PRC_52_hours_ago',
 'RTM_10_LMP_PRC_25_hours_ago',
 'RTM_10_LMP_PRC_23_hours_ago',
 'GAP LMP_PRC_41_hours_ago',
 'GAP LMP_PRC_14_hours_ago',
 'RTM_6_LMP_PRC_59_hours_ago',
 'RTM_6_LMP_PRC_19_hours_ago',
 'DAM LMP_PRC_34_hours_ago',
 'RTM_12_LMP_PRC_31_hours_ago',
 'RTM_11_LMP_PRC_50_hours_ago',
 'cloud_layer_1_code_set_1',
 'RTM_7_LMP_PRC_35_hours_ago',
 'RTM_2_LMP_PRC_22_hours_ago',
 'RTM_4_LMP_PRC_14_hours_ago',
 'RTM_4_LMP_PRC_59_hours_ago',
 'RTM_12_LMP_PRC_34_hours_ago',
 'RTM_2_LMP_PRC_21_hours_ago',
 'RTM_2_LMP_PRC_49_hours_ago',
 'RTM_8_LMP_PRC_16_hours_ago',
 'DNI',
 'RTM_3_LMP_PRC_39_hours_ago',
 'RTM_7_LMP_PRC_56_hours_ago',
 'RTM_5_LMP_PRC_13_hours_ago',
 'RTM_12_LMP_PRC_52_hours_ago',
 'RTM_9_LMP_PRC_36_hours_ago',
 'RTM_12_LMP_PRC_29_hours_ago',
 'RTM_5_LMP_PRC_46_hours_ago',
 'RTM_5_LMP_PRC_41_hours_ago',
 'RTM_1_LMP_PRC_47_hours_ago',
 'GAP LMP_PRC_32_hours_ago',
 'RTM_7_LMP_PRC_47_hours_ago',
 'RTM_7_LMP_PRC_50_hours_ago',
 'RTM_2_LMP_PRC_51_hours_ago',
 'RTM_5_LMP_PRC_39_hours_ago',
 'RTM_2_LMP_PRC_48_hours_ago',
 'Demand_Forecast_7Day_Ahead',
 'RTM_7_LMP_PRC_39_hours_ago',
 'RTM_4_LMP_PRC_49_hours_ago',
 ('DAM RT_Forecast_RTPD', 1),
 ('RT_Forecast_RTD', 7),
 'RTM_4_LMP_PRC_18_hours_ago',
 'RTM_12_LMP_PRC_24_hours_ago',
 'RTM_2_LMP_PRC_13_hours_ago',
 'RTM_9_LMP_PRC_13_hours_ago',
 'RTM_10_LMP_PRC_57_hours_ago',
 'RTM_9_LMP_PRC_24_hours_ago',
 'RTM_4_LMP_PRC_50_hours_ago',
 'DAM LMP_PRC_11_hours_ago',
 'RTM_9_LMP_PRC_42_hours_ago',
 'RTM_5_LMP_PRC_52_hours_ago',
 'RTM_6_LMP_PRC_40_hours_ago',
 'RTM_12_LMP_PRC_16_hours_ago',
 'cloud_layer_2_code_set_1',
 'RTM_8_LMP_PRC_58_hours_ago',
 'RTM_11_LMP_PRC_26_hours_ago',
 'DAM LMP_PRC_36_hours_ago',
 'DAM LMP_PRC_28_hours_ago',
 'hr_index',
 'RTM_2_LMP_PRC_12_hours_ago',
 'RTM_2_LMP_PRC_14_hours_ago',
 'RTM_7_LMP_PRC_17_hours_ago',
 'RTM_2_LMP_PRC_40_hours_ago',
 'RTM_9_LMP_PRC_14_hours_ago',
 'RTM_9_LMP_PRC_40_hours_ago',
 'RTM_10_LMP_PRC_45_hours_ago',
 'RTM_1_LMP_PRC_20_hours_ago',
 'RTM_9_LMP_PRC_37_hours_ago',
 'RTM_5_LMP_PRC_55_hours_ago',
 'DAM LMP_PRC_26_hours_ago',
 'RTM_12_LMP_PRC_35_hours_ago',
 'RTM_5_LMP_PRC_23_hours_ago',
 'RTM_12_LMP_PRC_30_hours_ago',
 'RTM_2_LMP_PRC_28_hours_ago',
 'RTM_3_LMP_PRC_48_hours_ago',
 'RTM_3_LMP_PRC_45_hours_ago',
 'RTM_8_LMP_PRC_31_hours_ago',
 'RTM_6_LMP_PRC_25_hours_ago',
 'RTM_3_LMP_PRC_29_hours_ago',
 'RTM_9_LMP_PRC_16_hours_ago',
 'GAP LMP_PRC_13_hours_ago',
 'RTM_3_LMP_PRC_32_hours_ago',
 'RTM_6_LMP_PRC_51_hours_ago',
 'RTM_10_LMP_PRC_48_hours_ago',
 'RTM_8_LMP_PRC_18_hours_ago',
 'RTM_8_LMP_PRC_35_hours_ago',
 'RTM_4_LMP_PRC_19_hours_ago',
 'RTM_7_LMP_PRC_29_hours_ago',
 'RTM_9_LMP_PRC_45_hours_ago',
 'RTM_3_LMP_PRC_47_hours_ago',
 'RTM_1_LMP_PRC_54_hours_ago',
 'RTM_6_LMP_PRC_32_hours_ago',
 'RTM_8_LMP_PRC_51_hours_ago',
 'RTM_10_LMP_PRC_21_hours_ago',
 'RTM_2_LMP_PRC_38_hours_ago',
 'RTM_4_LMP_PRC_26_hours_ago',
 'RTM_6_LMP_PRC_50_hours_ago',
 'RTM_3_LMP_PRC_24_hours_ago',
 'RTM_11_LMP_PRC_59_hours_ago',
 'DAM LMP_PRC_43_hours_ago',
 'RTM_8_LMP_PRC_48_hours_ago',
 'RTM_2_LMP_PRC_32_hours_ago',
 'RTM_11_LMP_PRC_19_hours_ago',
 'RTM_5_LMP_PRC_22_hours_ago',
 'RTM_10_LMP_PRC_26_hours_ago',
 'RTM_1_LMP_PRC_51_hours_ago',
 'GAP LMP_PRC_52_hours_ago',
 'RTM_8_LMP_PRC_59_hours_ago',
 'RTM_12_LMP_PRC_28_hours_ago',
 'RTM_6_LMP_PRC_29_hours_ago',
 'DAM LMP_PRC_3_hours_ago',
 'RTM_1_LMP_PRC_57_hours_ago',
 'RTM_10_LMP_PRC_40_hours_ago',
 'GAP LMP_PRC_39_hours_ago',
 'RTM_4_LMP_PRC_56_hours_ago',
 'RTM_10_LMP_PRC_13_hours_ago',
 'RTM_1_LMP_PRC_37_hours_ago',
 'RTM_9_LMP_PRC_52_hours_ago',
 'RTM_5_LMP_PRC_56_hours_ago',
 'RTM_2_LMP_PRC_55_hours_ago',
 'RTM_1_LMP_PRC_35_hours_ago',
 'RTM_2_LMP_PRC_45_hours_ago',
 'RTM_1_LMP_PRC_48_hours_ago',
 'RTM_6_LMP_PRC_45_hours_ago',
 'RTM_11_LMP_PRC_44_hours_ago',
 'RTM_6_LMP_PRC_27_hours_ago',
 'RTM_4_LMP_PRC_52_hours_ago',
 'RTM_11_LMP_PRC_57_hours_ago',
 'RTM_3_LMP_PRC_46_hours_ago',
 'GAP LMP_PRC_22_hours_ago',
 'RTM_1_LMP_PRC_42_hours_ago',
 'RTM_10_LMP_PRC_28_hours_ago',
 'DAM LMP_PRC_37_hours_ago',
 'RTM_8_LMP_PRC_32_hours_ago',
 'RTM_7_LMP_PRC_15_hours_ago',
 'RTM_6_LMP_PRC_46_hours_ago',
 'altimeter_set_1',
 'RTM_1_LMP_PRC_12_hours_ago',
 'RTM_4_LMP_PRC_33_hours_ago',
 'RTM_7_LMP_PRC_36_hours_ago',
 'RTM_3_LMP_PRC_50_hours_ago',
 'RTM_11_LMP_PRC_28_hours_ago',
 'RTM_2_LMP_PRC_26_hours_ago',
 'DAM LMP_PRC_7_hours_ago',
 'RTM_8_LMP_PRC_30_hours_ago',
 'RTM_2_LMP_PRC_29_hours_ago',
 'GAP LMP_PRC_27_hours_ago',
 'RTM_5_LMP_PRC_19_hours_ago',
 'DAM LMP_PRC_30_hours_ago',
 'GAP LMP_PRC_35_hours_ago',
 'RTM_12_LMP_PRC_56_hours_ago',
 'RTM_12_LMP_PRC_14_hours_ago',
 'RTM_2_LMP_PRC_20_hours_ago',
 'RTM_10_LMP_PRC_43_hours_ago',
 'RTM_6_LMP_PRC_55_hours_ago',
 'DAM LMP_PRC_35_hours_ago',
 'RTM_12_LMP_PRC_20_hours_ago',
 'RTM_9_LMP_PRC_18_hours_ago',
 'DAM LMP_PRC_8_hours_ago',
 'RTM_5_LMP_PRC_25_hours_ago',
 ('RT_Forecast_RTD', 11),
 'RTM_8_LMP_PRC_45_hours_ago',
 'RTM_1_LMP_PRC_18_hours_ago',
 'DAM LMP_PRC_40_hours_ago',
 'GAP LMP_PRC_31_hours_ago',
 'RTM_10_LMP_PRC_41_hours_ago',
 'RTM_12_LMP_PRC_15_hours_ago',
 'RTM_3_LMP_PRC_57_hours_ago',
 'RTM_3_LMP_PRC_49_hours_ago',
 'RTM_1_LMP_PRC_23_hours_ago',
 'RTM_1_LMP_PRC_19_hours_ago',
 'RTM_9_LMP_PRC_26_hours_ago',
 'RTM_4_LMP_PRC_51_hours_ago',
 'RTM_2_LMP_PRC_41_hours_ago',
 'RTM_1_LMP_PRC_40_hours_ago',
 'RTM_3_LMP_PRC_17_hours_ago',
 'RTM_11_LMP_PRC_49_hours_ago',
 'RTM_12_LMP_PRC_18_hours_ago',
 'RTM_7_LMP_PRC_55_hours_ago',
 ('RT_Forecast_RTD', 5),
 'RTM_2_LMP_PRC_52_hours_ago',
 'RTM_1_LMP_PRC_29_hours_ago',
 'RTM_3_LMP_PRC_14_hours_ago',
 'RTM_11_LMP_PRC_58_hours_ago',
 'RTM_10_LMP_PRC_46_hours_ago',
 'RTM_11_LMP_PRC_13_hours_ago',
 'RTM_12_LMP_PRC_23_hours_ago',
 'RTM_5_LMP_PRC_34_hours_ago',
 'RTM_11_LMP_PRC_34_hours_ago',
 'RTM_6_LMP_PRC_43_hours_ago',
 'RTM_7_LMP_PRC_13_hours_ago',
 'RTM_8_LMP_PRC_44_hours_ago',
 'RTM_6_LMP_PRC_28_hours_ago',
 'RTM_7_LMP_PRC_41_hours_ago',
 'RTM_8_LMP_PRC_38_hours_ago',
 'DAM LMP_PRC_14_hours_ago',
 'GAP LMP_PRC_40_hours_ago',
 'RTM_7_LMP_PRC_49_hours_ago',
 'RTM_5_LMP_PRC_24_hours_ago',
 'RTM_6_LMP_PRC_37_hours_ago',
 'RTM_4_LMP_PRC_36_hours_ago',
 'RTM_5_LMP_PRC_40_hours_ago',
 'RTM_12_LMP_PRC_58_hours_ago',
 'RTM_11_LMP_PRC_42_hours_ago',
 'RTM_2_LMP_PRC_37_hours_ago',
 'RTM_3_LMP_PRC_27_hours_ago',
 'RTM_1_LMP_PRC_30_hours_ago',
 'RTM_4_LMP_PRC_39_hours_ago',
 'RTM_4_LMP_PRC_44_hours_ago',
 'RTM_8_LMP_PRC_40_hours_ago',
 'Month',
 'RTM_6_LMP_PRC_49_hours_ago',
 'GAP LMP_PRC_36_hours_ago',
 'RTM_1_LMP_PRC_14_hours_ago',
 'GAP LMP_PRC_19_hours_ago',
 'RTM_8_LMP_PRC_36_hours_ago',
 'RTM_7_LMP_PRC_59_hours_ago',
 'RTM_12_LMP_PRC_55_hours_ago',
 'RTM_11_LMP_PRC_30_hours_ago',
 'RTM_5_LMP_PRC_53_hours_ago',
 'RTM_4_LMP_PRC_38_hours_ago',
 'RTM_7_LMP_PRC_31_hours_ago',
 'DAM LMP_PRC_31_hours_ago',
 'DAM LMP_PRC_18_hours_ago',
 'RTM_6_LMP_PRC_35_hours_ago',
 'DAM LMP_PRC_38_hours_ago',
 'RTM_4_LMP_PRC_13_hours_ago',
 'RTM_7_LMP_PRC_18_hours_ago',
 'DAM LMP_PRC_27_hours_ago',
 'Clearsky DHI',
 'RTM_10_LMP_PRC_44_hours_ago',
 'RTM_5_LMP_PRC_59_hours_ago',
 'RTM_4_LMP_PRC_45_hours_ago',
 'RTM_8_LMP_PRC_26_hours_ago',
 'RTM_5_LMP_PRC_58_hours_ago',
 'RTM_6_LMP_PRC_39_hours_ago',
 'RTM_5_LMP_PRC_30_hours_ago',
 'RTM_9_LMP_PRC_34_hours_ago',
 'RTM_6_LMP_PRC_20_hours_ago',
 'RTM_12_LMP_PRC_43_hours_ago',
 'RTM_2_LMP_PRC_18_hours_ago',
 'RTM_8_LMP_PRC_22_hours_ago',
 'RTM_12_LMP_PRC_27_hours_ago',
 'RTM_1_LMP_PRC_52_hours_ago',
 ('RT_Forecast_RTD', 12),
 'RTM_3_LMP_PRC_20_hours_ago',
 'RTM_1_LMP_PRC_49_hours_ago',
 'RTM_6_LMP_PRC_12_hours_ago',
 'RTM_5_LMP_PRC_26_hours_ago',
 'RTM_8_LMP_PRC_37_hours_ago',
 'RTM_8_LMP_PRC_15_hours_ago',
 'RTM_4_LMP_PRC_28_hours_ago',
 'RTM_1_LMP_PRC_50_hours_ago',
 'RTM_10_LMP_PRC_19_hours_ago',
 'RTM_4_LMP_PRC_43_hours_ago',
 'RTM_6_LMP_PRC_36_hours_ago',
 'RTM_5_LMP_PRC_31_hours_ago',
 'RTM_4_LMP_PRC_37_hours_ago',
 'RTM_5_LMP_PRC_15_hours_ago',
 'RTM_5_LMP_PRC_42_hours_ago',
 'RTM_3_LMP_PRC_53_hours_ago',
 'RTM_11_LMP_PRC_37_hours_ago',
 'RTM_10_LMP_PRC_33_hours_ago',
 'RTM_5_LMP_PRC_54_hours_ago',
 'RTM_3_LMP_PRC_30_hours_ago',
 'RTM_6_LMP_PRC_44_hours_ago',
 'RTM_11_LMP_PRC_56_hours_ago',
 'RTM_7_LMP_PRC_32_hours_ago',
 'RTM_9_LMP_PRC_32_hours_ago',
 'RTM_1_LMP_PRC_16_hours_ago',
 'RTM_11_LMP_PRC_47_hours_ago',
 ('DAM RT_Forecast_RTPD', 3),
 'RTM_7_LMP_PRC_16_hours_ago',
 'RTM_4_LMP_PRC_48_hours_ago',
 'RTM_12_LMP_PRC_32_hours_ago',
 'RTM_11_LMP_PRC_25_hours_ago',
 'RTM_3_LMP_PRC_54_hours_ago',
 'RTM_5_LMP_PRC_27_hours_ago',
 'RTM_2_LMP_PRC_27_hours_ago',
 'RTM_9_LMP_PRC_21_hours_ago',
 'RTM_11_LMP_PRC_41_hours_ago',
 'RTM_2_LMP_PRC_53_hours_ago',
 'DAM LMP_PRC_33_hours_ago',
 'RTM_10_LMP_PRC_17_hours_ago',
 'RTM_11_LMP_PRC_32_hours_ago',
 'RTM_9_LMP_PRC_41_hours_ago',
 'DAM LMP_PRC_16_hours_ago',
 'RTM_6_LMP_PRC_16_hours_ago',
 'RTM_10_LMP_PRC_15_hours_ago',
 'RTM_4_LMP_PRC_27_hours_ago',
 'RTM_7_LMP_PRC_28_hours_ago',
 'RTM_2_LMP_PRC_57_hours_ago',
 'RTM_5_LMP_PRC_37_hours_ago',
 'RTM_5_LMP_PRC_48_hours_ago',
 'RTM_11_LMP_PRC_20_hours_ago',
 'RTM_6_LMP_PRC_58_hours_ago',
 'RTM_3_LMP_PRC_36_hours_ago',
 'RTM_3_LMP_PRC_15_hours_ago',
 'RTM_1_LMP_PRC_28_hours_ago',
 'RTM_4_LMP_PRC_29_hours_ago',
 'RTM_11_LMP_PRC_33_hours_ago',
 'RTM_9_LMP_PRC_27_hours_ago',
 'RTM_9_LMP_PRC_43_hours_ago',
 'RTM_8_LMP_PRC_34_hours_ago',
 'RTM_11_LMP_PRC_39_hours_ago',
 'RTM_6_LMP_PRC_53_hours_ago',
 'wind_gust_set_1',
 'RTM_3_LMP_PRC_55_hours_ago',
 'RTM_8_LMP_PRC_47_hours_ago',
 'RTM_2_LMP_PRC_47_hours_ago',
 'RTM_1_LMP_PRC_45_hours_ago',
 'RTM_1_LMP_PRC_55_hours_ago',
 'RTM_5_LMP_PRC_18_hours_ago',
 'RTM_5_LMP_PRC_38_hours_ago',
 'RTM_2_LMP_PRC_42_hours_ago',
 'RTM_11_LMP_PRC_24_hours_ago',
 'RTM_2_LMP_PRC_17_hours_ago',
 'RTM_3_LMP_PRC_22_hours_ago',
 'RTM_2_LMP_PRC_56_hours_ago',
 'RTM_2_LMP_PRC_34_hours_ago',
 'RTM_6_LMP_PRC_54_hours_ago',
 'RTM_3_LMP_PRC_12_hours_ago',
 'RTM_11_LMP_PRC_54_hours_ago',
 'RTM_3_LMP_PRC_38_hours_ago',
 'RTM_8_LMP_PRC_56_hours_ago',
 'RTM_4_LMP_PRC_31_hours_ago',
 'RTM_1_LMP_PRC_36_hours_ago',
 'RTM_3_LMP_PRC_13_hours_ago',
 'RTM_11_LMP_PRC_51_hours_ago',
 'RTM_8_LMP_PRC_23_hours_ago',
 'RTM_3_LMP_PRC_44_hours_ago',
 'RTM_9_LMP_PRC_49_hours_ago',
 'RTM_10_LMP_PRC_37_hours_ago',
 'RTM_11_LMP_PRC_36_hours_ago',
 'RTM_7_LMP_PRC_20_hours_ago',
 'RTM_6_LMP_PRC_33_hours_ago',
 'RTM_1_LMP_PRC_39_hours_ago',
 'RTM_5_LMP_PRC_43_hours_ago',
 'RTM_9_LMP_PRC_58_hours_ago',
 'RTM_8_LMP_PRC_21_hours_ago',
 'RTM_7_LMP_PRC_38_hours_ago',
 ('RT_Forecast_RTD', 9),
 'RTM_8_LMP_PRC_33_hours_ago',
 'RTM_2_LMP_PRC_58_hours_ago',
 'Clearsky DNI',
 'RTM_1_LMP_PRC_22_hours_ago',
 'RTM_2_LMP_PRC_16_hours_ago',
 'RTM_9_LMP_PRC_20_hours_ago',
 'RTM_3_LMP_PRC_16_hours_ago',
 'RTM_4_LMP_PRC_30_hours_ago',
 'RTM_2_LMP_PRC_19_hours_ago',
 'RTM_1_LMP_PRC_41_hours_ago',
 'RTM_6_LMP_PRC_48_hours_ago',
 'RTM_5_LMP_PRC_47_hours_ago',
 'RTM_9_LMP_PRC_48_hours_ago',
 'RTM_10_LMP_PRC_49_hours_ago',
 'RTM_7_LMP_PRC_48_hours_ago',
 'RTM_4_LMP_PRC_15_hours_ago',
 'RTM_4_LMP_PRC_58_hours_ago',
 ('DAM RT_Forecast_RTPD', 4),
 'DAM LMP_PRC_32_hours_ago',
 'RTM_7_LMP_PRC_37_hours_ago',
 'RTM_11_LMP_PRC_35_hours_ago',
 'RTM_12_LMP_PRC_59_hours_ago',
 'RTM_12_LMP_PRC_12_hours_ago',
 'RTM_9_LMP_PRC_59_hours_ago',
 'RTM_9_LMP_PRC_15_hours_ago',
 'RTM_12_LMP_PRC_51_hours_ago',
 'RTM_7_LMP_PRC_58_hours_ago',
 'RTM_3_LMP_PRC_28_hours_ago',
 'RTM_12_LMP_PRC_36_hours_ago',
 'RTM_4_LMP_PRC_34_hours_ago',
 'RTM_10_LMP_PRC_52_hours_ago',
 'RTM_9_LMP_PRC_29_hours_ago',
 'RTM_11_LMP_PRC_40_hours_ago',
 'RTM_10_LMP_PRC_42_hours_ago',
 'RTM_12_LMP_PRC_53_hours_ago',
 'RTM_3_LMP_PRC_18_hours_ago',
 'RTM_6_LMP_PRC_41_hours_ago',
 'RTM_6_LMP_PRC_22_hours_ago',
 'RTM_5_LMP_PRC_50_hours_ago',
 'GAP LMP_PRC_53_hours_ago',
 'RTM_5_LMP_PRC_16_hours_ago',
 'RTM_5_LMP_PRC_57_hours_ago',
 'RTM_4_LMP_PRC_57_hours_ago',
 'RTM_9_LMP_PRC_46_hours_ago',
 'RTM_6_LMP_PRC_52_hours_ago',
 'RTM_9_LMP_PRC_31_hours_ago',
 'DAM LMP_PRC_17_hours_ago',
 'RTM_8_LMP_PRC_20_hours_ago',
 'RTM_10_LMP_PRC_31_hours_ago',
 'RTM_5_LMP_PRC_12_hours_ago',
 'RTM_2_LMP_PRC_31_hours_ago',
 'RTM_7_LMP_PRC_51_hours_ago',
 'RTM_8_LMP_PRC_25_hours_ago',
 'RTM_8_LMP_PRC_46_hours_ago',
 'RTM_12_LMP_PRC_17_hours_ago',
 'RTM_11_LMP_PRC_53_hours_ago',
 'RTM_12_LMP_PRC_13_hours_ago',
 'RTM_6_LMP_PRC_42_hours_ago',
 'RTM_2_LMP_PRC_39_hours_ago',
 'RTM_7_LMP_PRC_19_hours_ago',
 'RTM_12_LMP_PRC_33_hours_ago',
 'RTM_8_LMP_PRC_19_hours_ago',
 'RTM_6_LMP_PRC_23_hours_ago',
 'RTM_3_LMP_PRC_34_hours_ago',
 'RTM_10_LMP_PRC_16_hours_ago',
 'RTM_4_LMP_PRC_35_hours_ago',
 'RTM_1_LMP_PRC_27_hours_ago',
 'RTM_1_LMP_PRC_59_hours_ago',
 'RTM_8_LMP_PRC_53_hours_ago',
 'RTM_9_LMP_PRC_28_hours_ago',
 'RTM_1_LMP_PRC_53_hours_ago',
 'RTM_6_LMP_PRC_30_hours_ago',
 'RTM_5_LMP_PRC_29_hours_ago',
 'RTM_7_LMP_PRC_12_hours_ago',
 'RTM_8_LMP_PRC_50_hours_ago',
 'RTM_6_LMP_PRC_34_hours_ago',
 'RTM_1_LMP_PRC_44_hours_ago',
 'RTM_12_LMP_PRC_46_hours_ago',
 'RTM_9_LMP_PRC_12_hours_ago',
 'RTM_5_LMP_PRC_28_hours_ago',
 'RTM_3_LMP_PRC_31_hours_ago',
 'RTM_8_LMP_PRC_12_hours_ago',
 'RTM_5_LMP_PRC_44_hours_ago',
 'RTM_4_LMP_PRC_22_hours_ago',
 'RTM_10_LMP_PRC_54_hours_ago',
 'RTM_10_LMP_PRC_51_hours_ago',
 'RTM_8_LMP_PRC_28_hours_ago',
 'DAM LMP_PRC_19_hours_ago',
 'RTM_6_LMP_PRC_38_hours_ago',
 'RTM_6_LMP_PRC_31_hours_ago',
 'RTM_10_LMP_PRC_12_hours_ago',
 'RTM_11_LMP_PRC_17_hours_ago',
 'RTM_2_LMP_PRC_35_hours_ago',
 'precipitation',
 'RTM_3_LMP_PRC_37_hours_ago',
 'RTM_8_LMP_PRC_43_hours_ago',
 'RTM_7_LMP_PRC_22_hours_ago',
 'RTM_4_LMP_PRC_40_hours_ago',
 'RTM_1_LMP_PRC_56_hours_ago',
 'RTM_2_LMP_PRC_36_hours_ago',
 'RTM_1_LMP_PRC_38_hours_ago',
 'RTM_8_LMP_PRC_52_hours_ago',
 'RTM_3_LMP_PRC_43_hours_ago',
 'RTM_6_LMP_PRC_15_hours_ago',
 'RTM_9_LMP_PRC_51_hours_ago',
 'Clearsky GHI',
 'RTM_8_LMP_PRC_42_hours_ago',
 'IsWeekend',
 'RTM_6_LMP_PRC_47_hours_ago',
 'RTM_11_LMP_PRC_45_hours_ago',
 'RTM_11_LMP_PRC_43_hours_ago',
 'DAM LMP_PRC_41_hours_ago',
 'RTM_3_LMP_PRC_41_hours_ago',
 'RTM_12_LMP_PRC_19_hours_ago',
 'DAM LMP_PRC_13_hours_ago',
 'RTM_1_LMP_PRC_31_hours_ago',
 'IsHoliday',
 'RTM_3_LMP_PRC_58_hours_ago',
 'RTM_9_LMP_PRC_38_hours_ago',
 'IsWeekday',
 'RTM_10_LMP_PRC_32_hours_ago',
 'RTM_11_LMP_PRC_12_hours_ago',
 'RTM_12_LMP_PRC_42_hours_ago',
 'RTM_7_LMP_PRC_30_hours_ago',
 'heat_index_set_1d',
 'RTM_3_LMP_PRC_52_hours_ago',
 'RTM_5_LMP_PRC_51_hours_ago',
 'RTM_9_LMP_PRC_56_hours_ago',
 'RTM_2_LMP_PRC_15_hours_ago',
 'RTM_4_LMP_PRC_12_hours_ago',
 'RTM_7_LMP_PRC_43_hours_ago',
 'RTM_12_LMP_PRC_50_hours_ago',
 'RTM_10_LMP_PRC_56_hours_ago',
 'weather_cond_code_set_1',
 'RTM_10_LMP_PRC_36_hours_ago',
 'RTM_10_LMP_PRC_58_hours_ago',
 'wind_chill_set_1d',
 'RTM_4_LMP_PRC_41_hours_ago',
 'visibility_set_1',
 'RTM_2_LMP_PRC_50_hours_ago',
 'RTM_7_LMP_PRC_40_hours_ago',
 'RTM_5_LMP_PRC_35_hours_ago',
 'RTM_7_LMP_PRC_42_hours_ago',
 'Year',
 'precipitation_6',
 'precip_accum_one_hour_set_1']

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

f_predictions = []


df = pd.read_pickle('df.pkl')
df = df.reset_index(drop=True)
df['Gap'] = df['DAM LMP_PRC'] - df[('RTM LMP_PRC', 1)]
df = add_hours_dam(df, 1)
df = add_hours_rtm(df, 12)
df = add_hours_gap(df, 12)
Y_cols = ['Gap'] 
X_cols = [col for col in feature_names if col not in Y_cols+[
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_ENE_PRC', 1),
('RTM LMP_ENE_PRC', 2),
('RTM LMP_ENE_PRC', 3),
('RTM LMP_ENE_PRC', 4),
('RTM LMP_ENE_PRC', 5),
('RTM LMP_ENE_PRC', 6),
('RTM LMP_ENE_PRC', 7),
('RTM LMP_ENE_PRC', 8),
('RTM LMP_ENE_PRC', 9),
('RTM LMP_ENE_PRC', 10),
('RTM LMP_ENE_PRC', 11),
('RTM LMP_ENE_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
('RTM LMP_LOSS_PRC', 1),
('RTM LMP_LOSS_PRC', 2),
('RTM LMP_LOSS_PRC', 3),
('RTM LMP_LOSS_PRC', 4),
('RTM LMP_LOSS_PRC', 5),
('RTM LMP_LOSS_PRC', 6),
('RTM LMP_LOSS_PRC', 7),
('RTM LMP_LOSS_PRC', 8),
('RTM LMP_LOSS_PRC', 9),
('RTM LMP_LOSS_PRC', 10),
('RTM LMP_LOSS_PRC', 11),
('RTM LMP_LOSS_PRC', 12),
('RTM LMP_PRC', 1),
('TM LMP_PRC', 2),
('RTM LMP_PRC', 3),
('RTM LMP_PRC', 4),
('RTM LMP_PRC', 5),
('RTM LMP_PRC', 6),
('RTM LMP_PRC', 7),
('RTM LMP_PRC', 8),
('RTM LMP_PRC', 9),
('RTM LMP_PRC', 10),
('RTM LMP_PRC', 11),
('RTM LMP_PRC', 12),
'DAM LMP_CONG_PRC',
'DAM LMP_GHG_PRC',
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
'DAM LMP_ENE_PRC',
'DAM LMP_LOSS_PRC',
'Actual_Demand',
('RTM LMP_PRC', 1),
'DAM LMP_PRC',]]
          
Y = df[Y_cols]
X = df[X_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10615, shuffle=False, random_state=42)

ss1 = MinMaxScaler(feature_range=(0, 1))
X_train = ss1.fit_transform(X_train)
X_test = ss1.transform(X_test)
ss2 = MinMaxScaler(feature_range=(0, 1))
Y_train = ss2.fit_transform(Y_train)
Y_test = ss2.transform(Y_test)


svr = SVR()


params = {'C': [1e3], 'gamma': [0.1],'kernel': ['rbf'], 
         'epsilon':[0.1, 0.01, 0.001, 0.0001, 0.00001]}



# cross validation

folds = 5
svr_model_cv = GridSearchCV(estimator = svr,                         
                        param_grid = params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

svr_model_cv.fit(X_train, Y_train)





In [5]:
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

f_predictions = []


df = pd.read_pickle('df.pkl')
df = df.reset_index(drop=True)
df['Gap'] = df['DAM LMP_PRC'] - df[('RTM LMP_PRC', 1)]
df = add_hours_dam(df, 1)
df = add_hours_rtm(df, 12)
df = add_hours_gap(df, 12)
Y_cols = ['Gap'] 
X_cols = [col for col in feature_names if col not in Y_cols+[
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_ENE_PRC', 1),
('RTM LMP_ENE_PRC', 2),
('RTM LMP_ENE_PRC', 3),
('RTM LMP_ENE_PRC', 4),
('RTM LMP_ENE_PRC', 5),
('RTM LMP_ENE_PRC', 6),
('RTM LMP_ENE_PRC', 7),
('RTM LMP_ENE_PRC', 8),
('RTM LMP_ENE_PRC', 9),
('RTM LMP_ENE_PRC', 10),
('RTM LMP_ENE_PRC', 11),
('RTM LMP_ENE_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
('RTM LMP_LOSS_PRC', 1),
('RTM LMP_LOSS_PRC', 2),
('RTM LMP_LOSS_PRC', 3),
('RTM LMP_LOSS_PRC', 4),
('RTM LMP_LOSS_PRC', 5),
('RTM LMP_LOSS_PRC', 6),
('RTM LMP_LOSS_PRC', 7),
('RTM LMP_LOSS_PRC', 8),
('RTM LMP_LOSS_PRC', 9),
('RTM LMP_LOSS_PRC', 10),
('RTM LMP_LOSS_PRC', 11),
('RTM LMP_LOSS_PRC', 12),
('RTM LMP_PRC', 1),
('RTM LMP_PRC', 2),
('RTM LMP_PRC', 3),
('RTM LMP_PRC', 4),
('RTM LMP_PRC', 5),
('RTM LMP_PRC', 6),
('RTM LMP_PRC', 7),
('RTM LMP_PRC', 8),
('RTM LMP_PRC', 9),
('RTM LMP_PRC', 10),
('RTM LMP_PRC', 11),
('RTM LMP_PRC', 12),
'DAM LMP_CONG_PRC',
'DAM LMP_GHG_PRC',
('HASP LMP_CONG_PRC', 1),
('HASP LMP_CONG_PRC', 2),
('HASP LMP_CONG_PRC', 3),
('HASP LMP_CONG_PRC', 4),
('HASP LMP_ENE_PRC', 1),
('HASP LMP_ENE_PRC', 2),
('HASP LMP_ENE_PRC', 3),
('HASP LMP_ENE_PRC', 4),
('HASP LMP_GHG_PRC', 1),
('HASP LMP_LOSS_PRC', 1),
('HASP LMP_LOSS_PRC', 2),
('HASP LMP_LOSS_PRC', 3),
('HASP LMP_LOSS_PRC', 4),
('HASP LMP_PRC', 1),
('HASP LMP_PRC', 2),
('HASP LMP_PRC', 3),
('HASP LMP_PRC', 4),
('RTM LMP_CONG_PRC', 1),
('RTM LMP_CONG_PRC', 2),
('RTM LMP_CONG_PRC', 3),
('RTM LMP_CONG_PRC', 4),
('RTM LMP_CONG_PRC', 5),
('RTM LMP_CONG_PRC', 6),
('RTM LMP_CONG_PRC', 7),
('RTM LMP_CONG_PRC', 8),
('RTM LMP_CONG_PRC', 9),
('RTM LMP_CONG_PRC', 10),
('RTM LMP_CONG_PRC', 11),
('RTM LMP_CONG_PRC', 12),
('RTM LMP_GHG_PRC', 1),
('RTM LMP_GHG_PRC', 7),
'DAM LMP_ENE_PRC',
'DAM LMP_LOSS_PRC',
'Actual_Demand',
('RTM LMP_PRC', 1),
'DAM LMP_PRC',]]
          
Y = df[Y_cols]
X = df[X_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10615, shuffle=False, random_state=42)

ss1 = MinMaxScaler(feature_range=(0, 1))
X_train = ss1.fit_transform(X_train)
X_test = ss1.transform(X_test)
ss2 = MinMaxScaler(feature_range=(0, 1))
Y_train = ss2.fit_transform(Y_train)
Y_test = ss2.transform(Y_test)


svr = SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.001, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr.fit(X_train,Y_train)

C:\Users\Nika\Anaconda3\envs\PricePrediction\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.001, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [6]:
scaled_predictions = svr.predict(X_test)

In [24]:
Evaluate_Scaled(Y_test, scaled_predictions)

rMAE 5.598438569423931%
rRMSE 8.668631854116716%
MAPE 6.050234953654307%


In [7]:
predictions = ss2.inverse_transform(scaled_predictions.reshape(-1, 1))

In [9]:
np.save('SVR_GAP.npy', predictions) # save predictions

In [26]:
Y_true = []
temp = ss2.inverse_transform(Y_test)
for item in temp:
    Y_true.append(item[0])

In [27]:
Evaluate(Y_true, predictions)

MAE 83.76390373360063
RMSE 129.70017177574792


In [ ]:

plt.figure(figsize=(15,8))

plt.plot(Y_true,
         label='test y-true',
         alpha = 0.8,
         color='grey')
plt.plot(predictions,
         label='test predictions',
         alpha = 0.5,
         color='goldenrod')
plt.legend()
plt.xlim(0, 600)
plt.title('Gap - SVR Test: preds vs true', fontsize=18)
# plt.savefig('../images/dam_RNN_12_test_pred_vs_true_zoomed.jpg', bbox_inches='tight')
plt.show();

In [7]:
predictions

NameError: name 'predictions' is not defined